In [1]:
from collections import namedtuple
import csv
import psycopg2
from razi.rdkit_postgresql.types import Mol
import sys
from mytables import Compound

from rdkit import Chem, rdBase
from rdkit.Chem import Draw

from sqlalchemy import create_engine, Column, Index, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker


print(sys.version_info)
print(f'RDKit version: {rdBase.rdkitVersion}')

sys.version_info(major=3, minor=6, micro=0, releaselevel='final', serial=0)
RDKit version: 2017.03.1


### 1. データベースに接続する

`create_engine`関数を用いてPostgreSQLに接続する。

In [ ]:
engine = create_engine('postgresql://postgres@db:5432/postgres')

### 2. テーブルの定義

複数のNotebookで利用ができるように化学構造情報を保存するテーブルは[mytables.py](http://0.0.0.0:8888/edit/tutorial/mytables.py)に保存し  
`from mytables import Compound`で読み込んでいる。以下で各コードを説明していく。

[SQLAlchemy](http://docs.sqlalchemy.org/en/latest/orm/tutorial.html)でテーブルの定義をする方法と同じように`Razi`でも定義することができる。  
その際カラムのデータ型として化学構造データを保存する`Mol`が追加されている。

最初に`create_engine`関数を用いてPostgreSQLに接続する。  
引数として`postgresql://ユーザー名:パスワード@接続先:ポート/データベース名`を与える。  
今回は`ユーザー名=postgres`, `パスワード=なし`, `接続先=db`, `ポート=5432`, `データベース名=postgres` であるので以下のようになる。

```
engine = create_engine('postgresql://postgres@db:5432/postgres')
Base = declarative_base(bind=engine)
```

PostgresSQLに接続したengineを引数とした`Base`クラスを作成する。  
この`Base`クラスを継承したクラスを新たに作ることで`SQLAlchemy`や`Razi`では`データベース内に存在するテーブルを定義していることになる。

実際に`Compound`という名前のテーブルを定義していってみる。

```
class Compound(Base):
    __tablename__ = 'compounds'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    structure = Column(Mol)
    atompair = Column(Bfp)
    torsion = Column(Bfp)
    morgan = Column(Bfp)
    
    __table_args__ = (
        Index('compounds_structure', 'structure',
                    postgresql_using='gist'),
        )
```

```
    __tablename__ = 'compounds'
```

初めに`__tablename__`attribute (JAVAでいうfield) にテーブルの名前`compounds`を定義している。


例えば今回の例では`id`, `name`, `structure`, `atompair`, `torsion`, `morgan`という名前のカラムを持つテーブル`compounds`を作成している。  
(`atompair`, `torsion`, `morgan`に関しては[Razi-tutorial2-japanese.ipynb]で説明するので以降では説明しない)  
attributeに初期値として`Column`クラスをあたえることで`compounds`テーブルのカラムとして定義することができる。  
さらに`Column`クラスのコンストラクタの引数に`Integer`, `String`, `Mol`を与えることで  
それぞれ整数値、文字列、化学構造を保存するカラムであると定義している。  
加えて`id`には引数として`primary_key=True`を与えることでPrimary key(主キー)であると定義している。

最初にテーブルを定義するクラスが継承する`Base`クラスを呼び出す。

In [ ]:
Base = declarative_base(bind=engine)

In [ ]:
class Compound(Base):
    __tablename__ = 'compounds'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    structure = Column(Mol)
    atompair = Column(Bfp)
    torsion = Column(Bfp)
    morgan = Column(Bfp)
    
    __table_args__ = (
        Index('compounds_structure', 'structure',
                    postgresql_using='gist'),
        )
    
    def __init__(self, name, structure):
        self.name = name
        self.structure = structure
        if isinstance(self.structure, str):
            mol = Chem.MolFromSmiles(self.structure)
        else:
            mol = self.structure
        self.atompair = atompairbv_fp(self.structure)
        self.torsion = torsionbv_fp(self.structure)
        self.morgan = morganbv_fp(self.structure, 2)
        
    def __repr__(self):
        if isinstance(self.structure, Chem.Mol):
            return '(%s) < %s >' % (self.name, Chem.MolToSmiles(self.structure))
        return '(%s) < %s >' % (self.name, self.structure)

### 2. テーブルの構築

1で定義したテーブルを実際に構築するには以下の関数を実行する。

In [ ]:
Base.metadata.create_all()

### 3. データの登録

CHEMBLのデータである`tutorial_compounds.txt`は以下のような形式で保存されている。

In [ ]:
!head -n3 tutorial_compounds.txt

このファイルからvalidなsmilesとそれに対応する`CHEMBL ID`そして新たな`ID`を割り当てる関数`read_chembldb`を以下のように作成した。

In [ ]:
Record = namedtuple('Record', 'chembl_id, smiles, inchi, inchi_key')


def read_chembldb(filepath, limit=0):
    with open(filepath, 'rt') as inputfile:
            reader = csv.reader(inputfile, delimiter='\t', skipinitialspace=True)
            #headerを飛ばす
            next(reader)

            for count, record in enumerate(map(Record._make, reader), start=1):
                smiles = record.smiles

                #特定の三重結合をRDKitで読めるように変換する。
                smiles = smiles.replace('=N#N','=[N+]=[N-]')
                smiles = smiles.replace('N#N=','[N-]=[N+]=')            

                #invalidなsmilesは読み込まない
                if not Chem.MolFromSmiles(smiles):
                    continue

                yield count, record.chembl_id, smiles
                if count == limit:
                    break

期待する通りに関数が動いているか確かめる。関数`read_chembldb`は引数`limit`の数だけデータを取り出す(デフォルトは全てのデータを取り出す)。  
今回は`limit=3`で行ってみる。

In [ ]:
for count, chembl_id, smiles in read_chembldb('tutorial_compounds.txt', limit=3):
    print(count, chembl_id, smiles)

期待する通りに関数が動いているようだ。

データベースに接続し、検索や登録するには`sessionmaker`を用いる。

In [ ]:
Session = sessionmaker(bind=engine)

データを実際に登録してみる。定義したテーブル`Compound` classから一般的なオブジェクト指向型プログラミングにおけるオブジェクトを作成し、  
sessionオブジェクトの`add`メソッドを用いて登録すれば良い。

In [ ]:
session = Session()
for count, chembl_id, smiles in read_chembldb('tutorial_compounds.txt', 10):
    compound = Compound(chembl_id, smiles)
    session.add(compound)
session.commit()
session.close()

また別の登録方法としてRDkitのMolオブジェクトを引数に与える方法がある。

In [ ]:
session = Session()
smiles = 'c1ccccc1Cl'
mol =Chem.MolFromSmiles(smiles)
compound = Compound('111111', mol)
session.add(compound)
session.commit()
session.close()